In [1]:
import os
original_dir = os.getcwd()
os.chdir(os.path.dirname(os.path.dirname(os.getcwd())))
exec(open('setup/default.py').read())

In [2]:
raw_df = pd.read_csv(os.getcwd()+'/data/raw_data.csv')

## 식단 
### 게시글 기반

In [3]:
# 필터링 
body_df = raw_df[raw_df.type=='body'].copy()
body_df.loc[:, 'body'] = body_df['body'].str.replace(r'\s+|=', '', regex=True) # body에 \s(공백 = 줄바꿈, 띄어쓰기, 탭) 모두 (+) 제거 
#body_df = body_df[[ "[COURSE" in i for i in body_df.body]] # [COURSE 로 시작하는 경우만 남기기 
body_df.loc[:, 'body'] = body_df['body'].str.replace(r'Kcal', 'kcal', regex=True) # 칼로리 다르게 표기 된것 정리 

In [4]:
# 오전/오후를 24시간 형식으로 변환
body_df['time'] = body_df['time'].str.replace('오전', 'AM').str.replace('오후', 'PM')

# time 열을 time 형식으로 변환
body_df['time'] = pd.to_datetime(body_df['time'], format='%Y년 %m월 %d일 %p %I:%M', errors='coerce')

# 일(day), 시간 (hour), 요일(day of week) 추출
body_df['day'] = body_df['time'].dt.date
body_df['hour'] = body_df['time'].dt.strftime('%H:%M') 
body_df['day_of_week'] = body_df['time'].dt.strftime('%A')


In [ ]:
# # 스페셜 메뉴인 경우 빼기 
# def in_braket (input_string) :
#     # 정규 표현식을 사용하여 첫 번째 꺾쇠 안의 텍스트 추출
#     pattern = r"<([^>]+)>"
#     match = re.search(pattern, input_string)
#     if match:
#         extracted_value = match.group(1)
#         cleaned_string = re.sub(pattern, '', input_string, 1)  # 첫 번째 꺾쇠만 제거
#     else:
#         extracted_value = np.nan 
#         cleaned_string = input_string
#     return extracted_value,cleaned_string

In [17]:
# 메뉴가 겹치는 코스가 있는지 보기 위함.
def jaccard_similarity(s1, s2, n=2):
    def ngrams(string, n):
        return set([string[i:i+n] for i in range(len(string) - n + 1)])
    
    ngrams1 = ngrams(s1, n)
    ngrams2 = ngrams(s2, n)
    
    intersection = ngrams1.intersection(ngrams2)
    union = ngrams1.union(ngrams2)
    
    return len(intersection) / len(union)

In [22]:
# 코스 별 특성
def extract_course_info(text):
    # . : 모든 단일 문자와 일치
    # * : 앞의 문자가 0번 이상 반복
    # ? : 비탐욕적(가능한 한 적은 문자를) 일치 (? 이 있으면 가장 먼저 나온 kcal에서 끊어주기)
    # () : 일치된 부분을 캡처하는 그룹 생성
    course_pattern = r'\[(COURSE([\d&/,/]+))\]\s*(.*?)\s*(\d*)kcal\(단백질:(\d*)g\)Na함량:(\d*)mg'
    matches = re.findall(course_pattern, text, re.DOTALL)
    
    data = []
    #print (matches)
    for match in matches:
        course_full = match[0]  # [COURSE1]
        course_numbers = [int(e) for e in re.split('\D+', match[1]) if e] # [1], [1,2]
        course_menu = match[2].strip().replace('\n', ' ').replace('  ', ' ')
        #course_special, course_menu = in_braket(course_menu)
        course_kcal = int(match[3]) if match[3] else np.nan
        course_protein = int(match[4]) if match[4] else np.nan
        course_na = int(match[5]) if match[5] else np.nan

        # 기존 코스의 메뉴와 동일한 메뉴의 코스인지?        
        found_similar = False
        
        for existing_course in data:
            existing_numbers, existing_menu, existing_kcal, existing_protein, existing_na = existing_course
            # 기존 코스에 해당 코스와 동일한 메뉴라면, 기존 데이터에 코스 번호만 추가 
            if jaccard_similarity(course_menu, existing_menu) >= 0.9:
                existing_numbers.extend(course_numbers)
                found_similar = True
                break

        # 아니라면 데아터 추가 
        if not found_similar:
            data.append([course_numbers, course_menu, course_kcal, course_protein, course_na])
    

    return data

In [23]:
# 각 행의 course 리스트를 개별 행으로 확장
courses_expanded = body_df.explode('courses')
# course가 NA 인 것 제외 (ex.코스요)
courses_expanded = courses_expanded[courses_expanded['courses'].notna()] 
# course 정보 분리
course_details = pd.DataFrame(courses_expanded['courses'].tolist(), columns=['course_no', 'course', 'course_kcal', 'course_protein', 'course_na'])#, 'course_special'])


In [26]:
# 원래의 데이터프레임과 course_details 병합
courses_df = pd.concat([courses_expanded.drop(columns=['courses']).reset_index(drop=True), course_details.reset_index(drop=True)], axis=1)
courses_df.to_csv(os.getcwd()+'/data/preproc/main/menu.csv', sep=',' ,na_rep='NaN',index = False)

## 품절 여부 
### 댓글 기반

In [84]:
# 관리자가 쓴 댓글만 추출
comment_df = raw_df[(raw_df.type=='comment') & 
                    (raw_df.writer.isin(['Kitchen B1','kitchen B1','B1_Kitchen','원지연','kitchen_B1']))].copy()
comment_df.loc[:, 'body'] = comment_df['body'].str.replace(r'\s+|=', '', regex=True)

# 시간 설정
# 오전/오후를 24시간 형식으로 변환
comment_df.loc[:, 'time'] = comment_df['time'].str.replace('오전', 'AM').str.replace('오후', 'PM')

# [참고] comment_df는 날짜 형식이 2가지로 남음. (ex. 2022년 9월 21일, 4월 18일 오후 1:48 )
format2 = '%m월 %d일 %p %I:%M'
comment_df.loc[:, 'time2'] = pd.to_datetime(comment_df['time'], format=format2, errors='coerce')
# time 열을 time 형식으로 변환
comment_df.loc[:, 'time2']= pd.to_datetime(comment_df['time2'], format='%Y년 %m월 %d일 %p %I:%M', errors='coerce')
# 일(day), 시간 (hour), 요일(day of week) 추출
comment_df.loc[:, 'hour'] = comment_df['time2'].dt.strftime('%H:%M') 
# 불필요한 열 제거
comment_df.drop(columns=['time2'], inplace=True)


In [85]:
soldout_df= comment_df[comment_df['body'].str.contains("품절되|품절로|품절입")] # 품절되었습니다, 품절로, 품절입니다

In [86]:
# [참고] "품절되|품절로|품절입"는 안들어가지만, 품절이 들어가는 것중에 놓친 부분이 있는지? 
# 없음. -> 품절 대체찬 관련 내용임. 
print(comment_df[(comment_df['body'].str.contains("품절"))&(~comment_df['body'].str.contains("품절되|품절로|품절입"))])

        post_no     type      writer                           body  \
141   926901281  comment  Kitchen B1          2코너품절대체찬두부쑥갓나물이제공됩니다.   
198   926901318  comment  Kitchen B1       1코너품절대체찬으로2코너에콩자반이제공됩니다.   
232   926901334  comment  Kitchen B1  중식1코너품절대체찬으로2코너에서반반콩자반제공드립니다.   
233   926901334  comment  Kitchen B1     2코너품절대체찬반반콩자반->군만두로제공드립니다.   
252   926901350  comment  Kitchen B1  중식2코너품절대체찬으로1코너에서김자반볶음이제공됩니다.   
272   926901363  comment  Kitchen B1    1코너품절대체찬으로2코너에서쥐어채조림이제공됩니다.   
378   926901435  comment  Kitchen B1      1코너품절대체찬으로2코너에서부꾸미가제공됩니다.   
398   926901445  comment  Kitchen B1    2코너품절대체찬으로1코너에서감자샐러드가제공됩니다.   
697   926901641  comment  Kitchen B1     품절추가찬으로1코너에서감자샐러드를제공해드립니다.   
808   926901705  comment  Kitchen B1   중식1코너품절찬으로2코스에서동그랑땡조림이제공됩니다.   
997   926901833  comment  Kitchen B1          1코너품절찬으로순두부샐러드가제공됩니다.   
1010  926901843  comment  Kitchen B1     1코너품절찬으로2코너에서고구마샐러드가제공됩니다.   
1797  926901130  comment  Kitchen B1                품절대체찬계란찜제공드립니다.   
1809  

In [95]:
def extract_numbers(text):
    # 품절 앞에 있는 숫자만 추출하기 위해 패턴 정의
    match = re.search(r'(.*)품절', text)
    if match:
        # 품절 앞의 텍스트에서 숫자 추출
        pattern = r'([,\d\\/\\&]+)\s*(?:코스|코너)'
        match2 = re.search(pattern, match.group(1))
        
        if match2:
            return [int(e) for e in re.split('\D+', match2.group(1)) if e]
        else:
            return None        


# 데이터프레임에 새로운 열 추가
soldout_df['soldout'] = soldout_df['body'].apply(extract_numbers)

/Users/user/first_pytorch/lib/python3.7/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()


In [27]:
# 아래 코드는 재사용이 어려워서 fade out
# # 패턴 정의
# pattern = r"(1코너|1코스|2코너|2코스)"

# # 먼저 나오는 패턴 추출 (동시에 2개가 나오는 경우도 있음. ex. 1코너 품절로, 2코너에 추가찬 제공합니다.)

# def classify(text):
#     match = re.search(pattern, text)
#     if match:
#         found = match.group(0) # 더 먼저 나오는 것 기준
#         if found in ["1코너", "1코스"]:
#             return 1
#         elif found in ["2코너", "2코스"]:
#             return 2
#     else:
#         return None
# # 데이터프레임에 새로운 열 추가
# soldout_df['soldout'] = soldout_df['body'].apply(classify)


/Users/user/first_pytorch/lib/python3.7/site-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [97]:
soldout_df.sort_values(by=['soldout']).to_csv(os.getcwd()+'/data/preproc/main/soldout.csv', sep=',' ,na_rep='NaN',index = False)